
## Libraries import

In [1]:
from pysent import AspectAnotator
import pysent.aspect_annotators.extrassifiers as extrassifiers
from pysent.data_structures import concat_results, transform_output

import pandas as pd

C:\Users\welki\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


No CUDA GPU found in your device
[2024-01-21 18:22:10] (2.3.4) PyABSA(2.3.4): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.






C:\Users\welki\AppData\Local\Programs\Python\Python311\Lib\multiprocessing\pool.py:268: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>


## Load and filter the PolEmo dataset

In [2]:
polemo = pd.read_csv("../data/polemo.csv", index_col=0)

In [3]:
polemo = polemo.loc[polemo['review'].str.len() < 375,["review"]].reset_index(drop=True)

## Annotate the filtered PolEmo dataset using pyabsa "extraclassifier"

In [4]:
polemo_reviews_list = polemo.review.to_list()

In [5]:
pyabsa_full = extrassifiers.PyabsaExtrassifier()

[2024-01-21 18:22:21] (2.3.4) Please specify the task code, e.g. from pyabsa import TaskCodeOption
[2024-01-21 18:22:22] (2.3.4) Downloading checkpoint:multilingual 
[2024-01-21 18:22:22] (2.3.4) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2024-01-21 18:22:22] (2.3.4) Checkpoint already downloaded, skip
[2024-01-21 18:22:22] (2.3.4) Load aspect extractor from checkpoints\ATEPC_MULTILINGUAL_CHECKPOINT
[2024-01-21 18:22:22] (2.3.4) config: checkpoints\ATEPC_MULTILINGUAL_CHECKPOINT\fast_lcf_atepc.config
[2024-01-21 18:22:22] (2.3.4) state_dict: checkpoints\ATEPC_MULTILINGUAL_CHECKPOINT\fast_lcf_atepc.state_dict
[2024-01-21 18:22:22] (2.3.4) model: None
[2024-01-21 18:22:22] (2.3.4) tokenizer: checkpoints\ATEPC_MULTILINGUAL_CHECKPOINT\fast_lcf_atepc.tokenizer
[2024-01-21 18:22:24] (2.3.4) Set Model Device: cpu
[2024-01-21 18:22:24] (2.3.4) Device Name: Unknown


C:\Users\welki\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.


In [6]:
annotator = AspectAnotator([pyabsa_full])

In [7]:
annotations = annotator.annotate(polemo_reviews_list)

preparing apc inference dataloader: 100%|█████████████████████████████████████████| 2489/2489 [00:16<00:00, 147.26it/s]
C:\Users\welki\AppData\Roaming\Python\Python311\site-packages\pyabsa\tasks\AspectTermExtraction\prediction\aspect_extractor.py:566: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
classifying aspect sentiments: 100%|███████████████████████████████████████████████████| 78/78 [11:10<00:00,  8.60s/it]


## Restructure output

In [8]:
df_out = transform_output(reviews=polemo_reviews_list,
                          annotations=annotations)

In [9]:
df_out.to_csv("../data/polemo_labelled.csv", index=False)

## Manual tests

In [10]:
df_out = pd.read_csv("../data/polemo_labelled.csv")

In [11]:
df_out.groupby(["label"]).count()

,review,aspect
label,,
negative,940,940
neutral,182,182
positive,1367,1367


In [12]:
df_out

,review,aspect,label
0,Na samym wejściu hotel śmierdzi . W pokojach j...,hotel,negative
1,Na samym wejściu hotel śmierdzi . W pokojach j...,pokojach,negative
2,Na samym wejściu hotel śmierdzi . W pokojach j...,dywan,negative
3,Na samym wejściu hotel śmierdzi . W pokojach j...,łazience,negative
4,Na samym wejściu hotel śmierdzi . W pokojach j...,Wyposażenie,negative
...,...,...,...
2484,"Hotel z bardzo dobrym jedzeniem , jak już się ...",Hotel,positive
2485,"Hotel z bardzo dobrym jedzeniem , jak już się ...",jedzeniem,positive
2486,"Hotel z bardzo dobrym jedzeniem , jak już się ...",Obiekt,positive
2487,"Hotel z bardzo dobrym jedzeniem , jak już się ...",miejscu,positive


### Analayze 10 sample aspects

In [13]:
for index, row in df_out.sample(20).iterrows():
    print("#################")
    print(row['review'])
    print("######")
    print(f"{row['aspect']} -> {row['label']}")

#################
nie polecam , leczyła m dziecko u Pani doktor 2 , 5 roku , straciła m multum kasy na nie potrzebne leki o mało nie doprowadziła mojego dziecka do astmy bo przez ten cały czas leczyła bez jakichkolwiek badań jak zmieniła m lekarza to był zszokowany stane dziecka regularnie leczonego stanowczo nie polecam
######
lekarza -> negative
#################
będę miał teraz 4 sem zajęć z koszykówki z tym prowadzącym ! jak dla mnie bomba ! obowiązkowa rozgrzewka i pomimo paru niedociągnieć w jego sędziowaniu na serio polecam ! co prawda na dobra ocene trzeba tez wykazać sie zaangażowaniem no i wysoką frekwencją jednak prowdzący na serio godny polecenia !
######
prowadzącym -> positive
#################
Polecam , kurs ciekawy : ) Na koniec wystąpienie przed kamerą ( na dowolny temat , nie dłuższe niż 3 - 4 min . ) , które jest później analizowane w 4 oczy z prowadzącą . Nauka wszystkiego co w komunikacji potrzebne . Są to ćwiczenia , więc trzeba się odzywać i coś robić , ale zajęc

dobrze - 18


zle - 2

### Analyze 100 sample whole reviews - detailed

In [14]:
sample_whole_reviews = df_out.loc[df_out.review.isin(df_out.sample(100).review.to_list()),:]

In [15]:
sample_whole_reviews.to_csv("sample_whole_reviews.csv", index=False)

In [16]:
for index, row in sample_whole_reviews.iterrows():
    print("#################")
    print(row['review'])
    print("######")
    print(f"{row['aspect']} -> {row['label']}")

#################
Była m wczoraj na pierwszej wizycie u doktora Ufnalewskiego . Sympatyczny , pomocny , wprowadza atmosferę spokoju . Zapomniała m jednak zapytać w jakim szpitalu pracuje , czy wogóle pracuje w szpitalu czy tylko prowadzi swój prywatny gabinet ? Gdyby któraś z Was wiedziała bardzo proszę o odpowiedź , z góry dziękuję .
######
doktora -> positive
#################
Była m wczoraj na pierwszej wizycie u doktora Ufnalewskiego . Sympatyczny , pomocny , wprowadza atmosferę spokoju . Zapomniała m jednak zapytać w jakim szpitalu pracuje , czy wogóle pracuje w szpitalu czy tylko prowadzi swój prywatny gabinet ? Gdyby któraś z Was wiedziała bardzo proszę o odpowiedź , z góry dziękuję .
######
atmosferę -> positive
#################
Mulak Wojciech , dr Fizyka I - cwiczenia Ocena : Gorąco polecam [ / b ] Zaliczenie - nie ma problemu , same schematy , bardzo czesto daje na kołach powtórkowych te same zadania co na pierwszym kole . Gorąco polecam naprawde .
######
Zaliczenie -> posit

### Basic characteristics of the prepared dataset

In [17]:
print(f"The prepared dataset consists of {polemo.shape[0]} reviews")
print(f"The prepared dataset consists of {df_out.shape[0]} aspects")

The prepared dataset consists of 992 reviews
The prepared dataset consists of 2489 aspects
